In [ ]:
import numpy as np
from scipy import signal

# Using convolutions with a kernel to calculate values for rules

We can efficiently compute several quantities required for the rules using convolutions with a 2D kernel. 

This is analogous to sliding a window over the grid, and computing a quantity for each centre cell. In the simplest case, we
can count the number of neighbours belonging to each species with the following kernel:

In [ ]:
KERNEL_COUNT_NEIGHBOURS = np.array(
    [[1, 1, 1], [1, 0, 1], [1, 1, 1]],
)

As the kernel moves over the grid, we multiply the aligning elements by the kernel weights, and return the weighted sum. In this case, we multiply each neighbour of the center cell by 1, and return the sum. This gives us the number of neighbours of a given species.

Below, "state" contains two binary grid, with each layer corresponding to a separate species. The value is $1$ if the species exists in a given cell, and $0$ otherwise.

Notice that by applying the kernel to this state, we obtain a new numpy array with the same shape, where each element specifies the number of neighbours of each species.

In [ ]:
state = np.array(
    [
        [[1, 1, 0], [1, 0, 0], [0, 0, 0]],
        [[0, 0, 1], [0, 0, 1], [0, 0, 0]],
    ]
)

signal.convolve(
    state,
    KERNEL_COUNT_NEIGHBOURS[None, ...],
    mode="same",
)

We can use a modified kernel to simulate a diffusion process, such as _nutrient diffusion_, seen below.

Roughly, the nutrient level of a cell is reduced by 10% as the contents diffuse to neighbouring cells (center cell in the kernel), and increases proportional to the neighbouring cell nutrient levels (outer cells):

In [ ]:
KERNEL_DIFFUSE = np.array(
    [[0.1 / 8, 0.1 / 8, 0.1 / 8], [0.1 / 8, 0.9, 0.1 / 8], [0.1 / 8, 0.1 / 8, 0.1 / 8]]
)

Applying this to a grid of cell nutrient levels, we observe a diffusion process:

In [ ]:
nutrients = np.array(
    [
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5],
    ]
)

prior_nutrient_level = nutrients.sum()
print(f"Prior nutrient level: {prior_nutrient_level}")

diffused_nutrients = signal.convolve2d(
    nutrients,
    KERNEL_DIFFUSE,
    boundary="fill",
    mode="same",
)

print(f"New nutrient level: {diffused_nutrients.sum()}")
diffused_nutrients

Note that at the boundary, the convolution treats missing cells as having nutrient level $0$. This leads to a net reduction in the total nutrient level.

To maintain a net-zero change in the nutrient level, we could distribute the dissipated nutrients evenly across all cells:

In [ ]:
nutrients = np.array(
    [
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5],
    ]
)

prior_nutrient_level = nutrients.sum()
print(f"Prior nutrient level: {prior_nutrient_level}")

diffused_nutrients = signal.convolve2d(
    nutrients, KERNEL_DIFFUSE, boundary="fill", mode="same"
)

dissipated = prior_nutrient_level - diffused_nutrients.sum()

corrected_nutrients = diffused_nutrients + dissipated / diffused_nutrients.size
print(f"New nutrient level: {corrected_nutrients.sum()}")
corrected_nutrients